In [0]:
!pip install PyDrive
!pip install rouge

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import torch
import torch.nn as nn
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import re
import unicodedata

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Data for training set = 10k dev set = 1k
download = drive.CreateFile({'id': '1yJB36Jj1jw3U0JwE3IwhBYDY3VI1oCGd'})
download.GetContentFile('train.p')
download = drive.CreateFile({'id': '1T6YxqByvjNlCSZrU7FVmdCYlxM0RHlgw'})
download.GetContentFile('dev.p')
download = drive.CreateFile({'id': '1vPOGkgumTOVQ4N84GBo8AAPeAgwfyMep'})
download.GetContentFile('index2word.p')
download = drive.CreateFile({'id': '1JRYv-RQqhX6-fdYVUmyZmP9tBOtZjp_w'})
download.GetContentFile('word2index.p')

In [0]:
# #Data for training set = 50k dev set = 5k
# download = drive.CreateFile({'id': '1xq4JlZoWZ8_eO5F6VNBQvk2Mvub1Etpb'})
# download.GetContentFile('trainL.p')
# download = drive.CreateFile({'id': '1G3YAtOtrfv3PswNSHyGm0SIZy60n95zU'})
# download.GetContentFile('devL.p')
# download = drive.CreateFile({'id': '1T7rj8WokLSEEbc7-nqH48mjSnwBt4hEq'})
# download.GetContentFile('testL.p')
# download = drive.CreateFile({'id': '1bZXADxMyEn-yWtneA1sxDouGpuce1LRh'})
# download.GetContentFile('index2wordL.p')
# download = drive.CreateFile({'id': '1IMnW7Oyq4q9Ho5z87MbmVwlODjzEvY9h'})
# download.GetContentFile('word2indexL.p')

In [0]:
train_dict = pickle.load( open( "train.p", "rb" ) )
dev_dict = pickle.load( open( "dev.p", "rb" ) )
index2word = pickle.load( open( "index2word.p", "rb" ) )
word2index = pickle.load( open( "word2index.p", "rb" ) )

In [0]:
len(index2word)
len(word2index)

205064

In [0]:
#Extractive summary for 100 data points
# download = drive.CreateFile({'id': '1N6K91ySpywAB9pSKVCoJZKBn84S-2UFx'})
# download.GetContentFile('extract_train100.p')
# train_dict = pickle.load( open( "extract_train100.p", "rb" ) )

In [0]:
train_dict = train_dict[:]
dev_dict = dev_dict[:]

In [0]:
train_dict_text = []
count = 0
for i, data in enumerate(train_dict):
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
#         if i==789:
#             print(s, summ)
    t['summary'] = summ
    
    train_dict_text.append(t)
print(len(train_dict_text))

dev_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    dev_dict_text.append(t)
print(len(dev_dict_text))

51511
5678


In [0]:
ind = 1
print(train_dict_text[ind]['summary'])
print(train_dict_text[ind]['text'])
print(index2word[20439])

advertise in the daily telegraph; product information and key facts for    advertisers .
published: 2:27pm gmt 26 nov 2009  information and key facts about the daily telegraph .  the daily telegraph is britain 's biggest-selling quality daily newspaper , and retains the broadsheet format .  between monday and friday , it features seperate , dedicated sport and business sections .  on saturday 's , the multi-section daily telegraph includes the following:  the award-winning telegraph magazine is also included with the daily telegraph on saturdays .  big-name writers and contributors include simon heffer , boris johnson , jeff randall , brian moore , james cracknell , benedict brogan and andrew pierce .  the daily telegraph offers greater advertising impact than any compact - not only because it has bigger pages than the compacts , but also because readers spend longer with each page than any other quality daily . so , by advertising in our paper , you’ll get greater standout and more ex

In [0]:
import nltk
import nltk.data
from nltk import tokenize
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pickle
import json, gzip
import time
import math
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import euclidean
nltk.download('punkt')

W0508 21:09:03.680411 139769490491264 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Instructions for updating:
Colocations handled automatically by placer.


W0508 21:09:19.746385 139769490491264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module(module_url)
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

In [0]:
ext_train_data = []
ext_dev_data = []

for i, data in enumerate(train_dict_text):
    if i%1000==0:
        print(i)
    start = time.time()
    text = data['text']
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    try:
        sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
        num_sent = sentences_embeddings.shape[0]
        k = round(math.sqrt(num_sent))
        clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
        cluster_centers = clustering.cluster_centers_
        center_dist = defaultdict(list)
        for ns in range(num_sent):
            label = clustering.labels_[ns]
            dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
            center_dist[label].append((dist, ns))

        final = []
        for labels in range(k):
            try:
                center_dist[labels].sort(key = lambda x: x[0])
                final.append(center_dist[labels][0][1])
            except:
                continue
        final.sort()
        ex = []
        for i in final:
            ex.append(sentences_list[i])
        extractive = (' ').join(ex)
        dict_ = {}
        dict_['text'] = extractive
        dict_['summary'] = data['summary']
        ext_train_data.append(dict_)
    except:
        print('error')
end = time.time()
print(end-start)

for i, data in enumerate(dev_dict_text):
    start = time.time()
    text = data['text']
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive
    dict_['summary'] = data['summary']
    ext_dev_data.append(dict_)
end = time.time()
print(end-start)

0
0.03819131851196289
0.035803794860839844


In [0]:
ext_train_data = sorted(ext_train_data, key = lambda i: len(i['text']), reverse = True)
# dev_dict = sorted(dev_dict, key = lambda i: len(i['text']), reverse = True)

In [0]:
ind = 1
print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])
# print(ext_dev_data[ind]['text'])
# print(ext_dev_data[ind]['summary'])

05/19/2014 at 12:00 am edt  the blue jays won the game , but this kid  a pint-sized lothario used some quick sleight of hand to convince his crush he 'd given her a foul ball at a  – but a look at the replay confirmed he 'd been planning ahead . or so it appeared . the real one he likely kept as a treasured memento of the day he went viral .
a pint-size lothario used quick sleight of hand to convince his crush he 'd given her a foul ball


In [0]:
from nltk.tokenize import word_tokenize
ind2word = {0: 'PAD', 1:'BOS', 2:'EOS', 3:'UNK'}
word2ind = {'PAD':0, 'BOS':1, 'EOS':2, 'UNK':3}
count = 4
for data in ext_train_data:
    text = data['text']
    summ = data['summary']
    for words in word_tokenize(text):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

    for words in word_tokenize(summ):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1
print(len(word2ind))
print(len(ind2word))

343
343


In [0]:
# train_maxlen = 0
# for i, data in enumerate(ext_train_data):
#     summ = data['summary']
#     summ_ind = [1,2]
#     for words in word_tokenize(summ):
#         summ_ind.append(word2ind[words])
#     if train_maxlen < len(summ_ind):
#         train_maxlen = len(summ_ind)

# dev_maxlen = 0
# for i, data in enumerate(ext_dev_data):
#     summ = data['summary']
#     summ_ind = [1,2]
#     for words in word_tokenize(summ):
#         if words in word2ind:
#             summ_ind.append(word2ind[words])
#         else:
#             summ_ind.append(word2ind['UNK'])
#     if dev_maxlen < len(summ_ind):
#         dev_maxlen = len(summ_ind)
# print(train_maxlen, dev_maxlen)

In [0]:
train_dict = []
dev_dict = []
train_maxlen = 30
dev_maxlen = 30

for i, data in enumerate(ext_train_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(summ):
        summ_ind.append(word2ind[words])
    summ_ind = [1]+summ_ind+[2]
    while len(summ_ind)<train_maxlen:
        summ_ind.append(0)
    if len(summ_ind)==train_maxlen:
        data_point['text'] = word_tokenize(text)
        data_point['text_len'] = len(data_point['text'])
        data_point['summary_ind'] = summ_ind[:]
        data_point['summary'] = (['BOS']+word_tokenize(summ)+['EOS'])[:]
        train_dict.append(data_point)

    
    
for i, data in enumerate(ext_dev_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    if len(summ_ind)<dev_maxlen:
        summ_ind += [0]*(dev_maxlen-len(summ_ind))
    
    if len(summ_ind)==dev_maxlen:
        data_point['text'] = word_tokenize(text)
        data_point['text_len'] = len(data_point['text'])
        data_point['summary_ind'] = summ_ind[:]
        data_point['summary'] = (['BOS']+word_tokenize(summ)+['EOS'])[:]
        dev_dict.append(data_point)

In [0]:
print(len(train_dict[0]['text']))
print(len(train_dict[0]['summary']))
print(len(train_dict[0]['summary_ind']))
print(train_dict[0]['text_len'])

64
12
30
64


In [0]:
data = {}
data['train'] = train_dict
data['dev'] = dev_dict
print(len(data['train']))
print(len(data['dev']))

10
10


In [0]:
!pip install allennlp
from allennlp.modules.elmo import Elmo
from allennlp.modules.elmo import batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 1, dropout=0, requires_grad = True)

In [0]:
# class EncoderLSTM(nn.Module):
    
#     def __init__(self, params):
#         super(EncoderLSTM, self).__init__()
#         self.hidden_size = params['hidden_size']
#         self.emb_size = params['emb_size']
#         self.vocab_size = params['vocab_size']
#         self.num_layers = params['num_layers']
#         self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
#         self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers)

#     def forward(self, input, hidden, memory, seq_lengths):
#         batch, seq_len = input.size()
#         character_ids = batch_to_ids(input).cuda()
#         emb = self.embedding(input)
#         packed_emb = pack_padded_sequence(emb, (seq_lengths).cpu().numpy(), batch_first = True)
#         packed_output, (h, c) = self.lstm(packed_emb, (hidden, memory))
#         output, _ = pad_packed_sequence(packed_output, batch_first = True)
#         return output, h, c

#     def initHidden(self, batch_size):
#         h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
#         c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
#         return h_init, c_init

In [0]:
class biEncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(biEncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers, bidirectional = True)
        self.elmo = Elmo(options_file, weight_file, 1, requires_grad=True, dropout=0)
        
    def forward(self, input, hidden, memory, seq_lengths):
        batch = len(input)
        character_ids = batch_to_ids(input).cuda()
        emb = (self.elmo(character_ids)['elmo_representations'][0]).cuda()
        output, (h, c) = self.lstm(emb, (hidden, memory))
        output1 = output[ :, :, :self.hidden_size]
        output2 = output[:, :, self.hidden_size:] #reverse
        output = output1+output2
        h = h.view(self.num_layers, 2, batch, self.hidden_size)
        c = c.view(self.num_layers, 2, batch, self.hidden_size)
        h = h[:,0,:,:]
        c = c[:,0,:,:]
        return output, h, c

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
# class DecoderLSTM(nn.Module):
    
#     def __init__(self, params):
#         super(DecoderLSTM, self).__init__()
#         self.hidden_size = params['hidden_size']
#         self.vocab_size = params['vocab_size']
#         self.emb_size = params['emb_size']
#         self.num_layers = params['num_layers']
#         self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
#         self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True)
#         self.w = nn.Linear(self.hidden_size, self.vocab_size)

#     def forward(self, input, hidden, memory):
#         batch_size, seq_len = input.size()
#         emb = self.embedding(input)
#         output, (h, c) = self.lstm(emb, (hidden, memory))
#         logits = self.w(output)
#         return logits, h, c

#     def initHidden(self):
#         h_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
#         c_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
#         return h_init, c_init

In [0]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.batch_size = params['batch_size']
        #self.dropout_p = params['dropout_p']

        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        #self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        #self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first=True, num_layers = self.num_layers)
        self.w = nn.Linear(2*self.hidden_size, self.vocab_size)
        self.elmo = Elmo(options_file, weight_file, 1, requires_grad=True, dropout=0)

    def forward(self, input, hidden, memory, encoder_outputs, pad_ind):
        batch_size = len(input)
        character_ids = batch_to_ids(input).cuda()
        #embedded = self.embedding(input)
        #embedded = self.dropout(embedded)
        embedded = (self.elmo(character_ids)['elmo_representations'][0]).cuda()        
        output,(h,c) = self.lstm(embedded, (hidden, memory))
        
        attn_values = torch.bmm(output,encoder_outputs.permute(0,2,1))
        dec_attn_values = torch.bmm(output,output.permute(0,2,1))
    
        for b in range(batch_size):
            attn_values[b,:,pad_ind[b]:] = -1e10
        
        mask = torch.ones_like(dec_attn_values).cuda()
        mask1 = -1e10*torch.ones_like(dec_attn_values).cuda()
        mask = torch.tril(mask, diagonal = -1)
        mask1 = torch.triu(mask1)
        mask = mask+mask1
        dec_attn_values = dec_attn_values*mask.float()
        
        total_attn_values = torch.cat((attn_values,dec_attn_values),2)
        total_outputs = torch.cat((encoder_outputs,output),1)
        
        sftmax = nn.Softmax(dim=2)
        total_attn_weights = sftmax(total_attn_values)
        weight_attn = torch.bmm(total_attn_weights, total_outputs)
        final_hidden = torch.cat((output,weight_attn),2)
        scores = self.w(final_hidden)
        return scores, h, c

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
def train_lm(dataset, params, net, fscore = False):
    
    #print(dataset[9920]['summary'])
    # since the first index corresponds to the PAD token, we just ignore it
    # when computing the loss
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    sftmax = nn.Softmax(dim=2)
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = len(dataset)    
    batches = [(start, start + params['batch_size']) for start in range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        #random.shuffle(batches)
        ref_summ = []
        gen_summ = []
        count = 0
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            
            batch_text = []
            batch_summary = []
            batch_pad_len = []
#             batch_summ_indexes = []
            batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
            i = 0
            for d in batch:
                
                batch_text.append(d['text'])
                batch_summary.append(d['summary'])
                batch_pad_len.append(d['text_len'])
#                 batch_summ_indexes.append(d['summary_ind'])
                batch_summ_indexes[i] = d['summary_ind']
                #print(len(d['summary_ind']), len(d['summary']))
                
                i += 1
            
            
            batch_summ_indexes = torch.from_numpy(batch_summ_indexes).long().cuda()
#             batch_summ_indexes = torch.tensor(batch_summ_indexes).long.cuda()
            pred = net(batch_text, batch_summary, batch_pad_len)
            sz = pred.size()[1]
            preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
            targets = batch_summ_indexes[:, 1:sz].contiguous().view(-1)
            #print(pred.size(),preds.size(),targets.size(),batch_summ_indexes.size())
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            
            if fscore:
                logits = sftmax(pred)
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summ_indexes[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summ_indexes[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='
                    if not gold or gold=='.'*len(gold):
                        continue

                    gen_summ.append(summ)
                    ref_summ.append(gold)
        print(start)   
        ind = 0
        scores = pred[ind,:,:].unsqueeze(0)
        logits = sftmax(scores)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(batch_summ_indexes[ind,1:].size()[0]):
            gold_summ_last += ind2word[batch_summ_indexes[ind,l].item()] + ' '
        for l in range(words.size()[0]):
            summ_last += ind2word[words[l].item()] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        #print(len(gen_summ), len(ref_summ))
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        
#         print('epoch: %d, loss: %0.2f, time: %0.2f sec' %\
#                 (epoch, ep_loss, time.time()-start_time))
        loss, perplex =  compute_loss(data['dev'], net, params['batch_size'], fscore = True)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev loss: %0.2f, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, loss, perplex))
        print('####################################################')

In [0]:
def compute_loss(dataset, net, bsz=10, fscore = False):
    
    sftmax= nn.Softmax(dim=2)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    gen_summ = []
    ref_summ = []
    count = 0
    for b_idx, (start, end) in enumerate(batches):
#         print(start, end)
        batch = dataset[start:end]
    
        batch_text = []
        batch_summary = []
        batch_pad_len = []
        batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
        i = 0
        for d in batch:
            batch_text.append(d['text'])
            batch_summary.append(d['summary'])
            batch_pad_len.append(d['text_len'])
            batch_summ_indexes[i] = d['summary_ind']
            i += 1
                
        #batch_text = torch.from_numpy(np.array(batch_text)).long().cuda()
        #batch_summary = torch.from_numpy(np.array(batch_summary)).long().cuda()
        batch_pad_len = torch.from_numpy(np.array(batch_pad_len)).long().cuda()
        batch_summ_indexes = torch.from_numpy(batch_summ_indexes).long().cuda()
#        ut = torch.nonzero(batch_text).size(0)
        pred = net(batch_text, batch_summary, batch_pad_len)
        preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
        sz = pred.size()[1]
        targets = batch_summ_indexes[:, 1:sz].contiguous().view(-1)
        loss = criterion(preds, targets)
        nll += loss.detach()
 #       total_unmasked_tokens += ut
        
        if fscore:
            #print('pred', pred.size())
            logits = sftmax(pred)
            words = torch.argmax(logits, dim=2)
            #print('words', words.size())
            for sent in range(words.size()[0]):
                summ = ''
                gold = ''
                for word in range(words.size()[1]):
                    if ind2word[words[sent][word].item()] == 'EOS':
                        break
                    if summ :
                        summ += ' '
                    summ += ind2word[words[sent][word].item()]

                for word in range(1,len(batch_summ_indexes[sent])-1):
                    if ind2word[batch_summ_indexes[sent][word].item()] == 'EOS':
                        break
                    if gold:
                        gold += ' '
                    gold += ind2word[batch_summ_indexes[sent][word].item()]

                if summ == '.'*len(summ):
                    count += 1
                if not summ or summ == '.'*len(summ):
                    summ = '====='
                if not gold or gold=='.'*len(gold):
                    continue

                gen_summ.append(summ)
                ref_summ.append(gold)
        
    ind = 0
    scores = pred[ind,:,:].unsqueeze(0)
    logits = sftmax(scores)
    words = torch.argmax(logits, dim=2).squeeze(0)
    gold_summ = ''
    summ = ''
    for l in range(batch_summ_indexes[ind,1:].size()[0]):
        gold_summ += ind2word[batch_summ_indexes[ind,l].item()] + ' '
    for l in range(words.size()[0]):
        summ += ind2word[words[l].item()] + ' '
            
    
    print('#############Dev###########')
    if fscore:
        rouge_score = calculate_rouge(gen_summ, ref_summ)
        print('rouge', rouge_score)
    print(gold_summ)
    print(summ)
    
    perplexity = torch.exp(nll).cpu() #torch.exp(nll / total_unmasked_tokens).cpu()
    return nll, perplexity.data


In [0]:
# # function to evaluate LM perplexity on some input data, DO NOT MODIFY
# def test_lm(dataset, net, bsz=1, beam_search = False, max_len = 22):
    
#     sftmax = nn.Softmax(dim=1)
#     criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
#     num_examples = len(dataset)  
#     # we'll still use batches b/c we can't fit the whole
#     # validation set into GPU memory
#     batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
#     nll = 0
#     ans = []
#     for b_idx, (start, end) in enumerate(batches):
            
#         batch = dataset[start:end]
#         batch_text = np.zeros([len(batch), len(batch[0]['text'])])
#         batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
#         batch_pad_len = np.zeros([len(batch)])
#         i = 0
        
#         for d in batch:
#             batch_text[i] = d['text']
#             batch_summary[i] = d['summary']
#             batch_pad_len[i] = d['text_len']
#             i += 1
            
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         batch_summary_gen = torch.ones([bsz, 1], dtype=torch.long).cuda()
        
#         for i in range(max_len):    
#             preds = net(batch_text, batch_summary_gen, batch_pad_len)[:,-1,:]
#             distribution = sftmax(preds)
#             ind = torch.argmax(distribution,1).cpu()
#             if(ind2word[ans[-1]] == "EOS"):
#                 break
# #             batch_summary_test = torch.cat((batch_summary_test,x),1) 
# #         print("Inside test_lm: ")
# #         print(*ans, sep = ' ')
# # #         targets = batch_summary[:, 1:].contiguous().view(-1)
# # #         preds = preds[:-1, :].contiguous().view(-1, net.vocab_size)
# # #         loss = criterion(preds, targets)
# # #         nll += loss.detach()

#     return nll

In [0]:
# # function to evaluate LM perplexity on some input data, DO NOT MODIFY
# def test_lm_bms(dataset, net, bsz=1):
    
#     net1 = net
#     net2 = net
    
#     criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
#     num_examples = len(dataset)  
    
#     batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
#     nll = 0
#     max_len = 22
#     for b_idx, (start, end) in enumerate(batches):
            
#         batch = dataset[start:end]
#         batch_text = np.zeros([len(batch), len(batch[0]['text'])])
#         batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
#         batch_pad_len = np.zeros([len(batch)])
#         i = 0
#         ans = []
#         for d in batch:
#             batch_text[i] = d['text']
#             batch_summary[i] = d['summary']
#             batch_pad_len[i] = d['text_len']
#             i += 1
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         gold_batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         dec_batch_summary1 = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         dec_batch_summary2 = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         for i in range(max_len):    
            
#             if dec_batch_summary1[0][-1]==2 and dec_batch_summary2[0][-1]==2:
#                 break
#             else:
#                 if dec_batch_summary1[0][-1]!=2:
#                     preds1 = net1(batch_text, dec_batch_summary1, batch_pad_len)
#                     preds1 = preds1.contiguous().view(-1, net.vocab_size)
#                     sftmax = nn.Softmax(dim=1)
#                     distribution1 = sftmax(preds1)
#                     prob1, top_words1 = torch.topk(distribution1[-1],k=2,dim=0)
#                     #print('1',prob1, top_words1)

#                 else:
#                     prob1 = -1e40*torch.ones_like(prob1)

#                 if dec_batch_summary2[0][-1]!=2:
#                     preds2 = net2(batch_text, dec_batch_summary2, batch_pad_len)
#                     preds2 = preds2.contiguous().view(-1, net.vocab_size)
#                     sftmax = nn.Softmax(dim=1)
#                     distribution2 = sftmax(preds2)
#                     prob2, top_words2 = torch.topk(distribution2[-1],k=2,dim=0)
#                     #print('2',prob2, top_words2)

#                 else:
#                     prob2 = -1e40*torch.ones_like(prob2)

#                 prob_cat = torch.cat((prob1, prob2))
#                 top_words_cat = torch.cat((top_words1, top_words2))
#                 final_top_words = torch.topk(prob_cat,k=2,dim=0)[1]
#                 #print(prob_cat, final_top_words)
#                 #print(dec_batch_summary1)

#                 if final_top_words[0] in [0,1]:
#                     dec_batch_summary1 = torch.cat((dec_batch_summary1,top_words1[final_top_words[0]].view(-1,1)),1) 
#                 elif final_top_words[0] in [2,3]:
#                     dec_batch_summary1 = torch.cat((dec_batch_summary2,top_words2[final_top_words[0]-2].view(-1,1)),1)
#                 if final_top_words[1] in [0,1]:
#                     dec_batch_summary2 = torch.cat((dec_batch_summary1,top_words1[final_top_words[1]].view(-1,1)),1) 
#                 elif final_top_words[0] in [2,3]:
#                     dec_batch_summary2 = torch.cat((dec_batch_summary2,top_words2[final_top_words[1]-2].view(-1,1)),1)
            
#             #print(dec_batch_summary1)
#             #print(ind2word[temp.item()])
#             #if(ind2word[temp.item()] == "EOS"):
#             #    break
#            # batch_summary_test = torch.cat((batch_summary_test,x),1) 
# #         targets = batch_summary[:, 1:].contiguous().view(-1)
# #         preds = preds[:-1, :].contiguous().view(-1, net.vocab_size)
# #         loss = criterion(preds, targets)
# #         nll += loss.detach()
    
#         summ_str1 = ''
#         summ_str2 = ''
#         for l in range(dec_batch_summary1.size()[1]):
#             summ_str1 += ind2word[dec_batch_summary1[0][l].item()]+' '
#         for l in range(dec_batch_summary2.size()[1]):
#             summ_str2 += ind2word[dec_batch_summary2[0][l].item()]+' '
    
#     return nll, summ_str1,summ_str2

In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def test_lm(dataset, net, bsz=1, beam_search = False, max_len = 30):
    
    sftmax = nn.Softmax(dim=1)
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    nll = 0
    ans = []
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = []
        batch_summary = []
        batch_pad_len = []
        batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
        i = 0
        
        for d in batch:
                batch_text.append(d['text'])
                batch_summary.append(d['summary'])
                batch_pad_len.append(d['text_len'])
                batch_summ_indexes[i] = d['summary_ind']
                i += 1
                
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         batch_summary_gen = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         bos = ['BOS']
        batch_summary_gen = [['BOS'] for i in range(bsz)]
        
        for i in range(max_len):    
            preds = net(batch_text, batch_summary_gen, batch_pad_len)[:,-1,:]
            distribution = sftmax(preds)
            ind = torch.argmax(distribution,1)
            for j in range(bsz):
              batch_summary_gen[j].append(ind2word[ind[j].item()])
        
        ans+=batch_summary_gen.copy()
    
    gen_summ = []
    gold_summ = []
    
    for n in range(num_examples):
        summ = ''
#         summ = (' ').join(ans[n])
        for k in ans[n][1:]:
          if k == 'EOS':
            break
          summ += k + ' '  
        gen_summ.append(summ)
        
    for n in range(num_examples):
        summ = ''
#         summ = (' ').join(dataset[n]['summary'])
        for k in dataset[n]['summary'][1:]:
          if k == 'EOS':
            break
          summ += k + ' '
        gold_summ.append(summ)
    print((gold_summ))
    print((gen_summ))
    rouge_score = calculate_rouge(gen_summ, gold_summ)
    print('rouge', rouge_score)

In [0]:
class SuperAwesome(nn.Module):
    def __init__(self, params):
        super(SuperAwesome, self).__init__()
        #self.enc = EncoderLSTM(params).cuda()
        self.bienc = biEncoderLSTM(params).cuda()
        #self.dc = DecoderLSTM(params).cuda()
        self.attn_dc = AttnDecoderLSTM(params).cuda()
        self.vocab_size = params['vocab_size']
    def forward(self, batch_text, batch_summary, batch_pad_len):
        batch_size = len(batch_text)
        
#         h_init,c_init = self.enc.initHidden(batch_size)
#         enc_output,h_enc,c_enc = self.enc(batch_text,h_init,c_init, batch_pad_len)
        
        h_init,c_init = self.bienc.initHidden(batch_size)
        enc_output,h_enc,c_enc = self.bienc(batch_text,h_init,c_init, batch_pad_len)
        #h_init,c_init = self.attn_dc.initHidden(batch_size)
        h_init, c_init = h_enc.contiguous(),c_enc.contiguous()
        attn_pre,h,c = self.attn_dc(batch_summary,h_init,c_init,enc_output, batch_pad_len)
#         attn_pre, h, c = self.dc(batch_summary, h_enc, c_enc)
        return attn_pre

In [0]:
params = {}
params['hidden_size'] = 200
params['emb_size'] = 1024
params['num_layers'] = 1
params['batch_size'] = 50
params['vocab_size'] = len(word2ind)
params['learning_rate'] = 0.01
params['epochs'] = 30
net = SuperAwesome(params)

train_lm(data['train'], params, net, fscore = True)

0
########################Train######################
BOS new vc effort will focus on capital-efficient energy companies . EOS PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
marissa pennsylvania portion former nbc at at at at at at at at at planning by by by by by by by 
rouge {'rouge-1': {'f': 0.02687746887733764, 'p': 0.030202020202020202, 'r': 0.024358974358974356}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.026151330768507335, 'p': 0.030202020202020202, 'r': 0.024358974358974356}}
#############Dev###########
rouge {'rouge-1': {'f': 0.04523809467715421, 'p': 0.15, 'r': 0.02909090909090909}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.030963670437366475, 'p': 0.15, 'r': 0.02909090909090909}}
BOS the UNK 's UNK was held at UNK UNK wednesday at his UNK UNK , UNK UNK to people EOS PAD PAD PAD PAD PAD PAD PAD PAD 
the EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS 
epoch: 0, loss: 5.86, time: 

In [0]:
net.eval()
test_lm(data['test'][0:20], net, bsz=10)